## How To: Use `Libgen` class to Download Books

In [13]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from autobook.libgen import Libgen

data = Libgen(q="the sun also rises")
df = data.search()
df = data.filter(author="Ernest Hemingway", title="The Sun Also Rises")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
df.get_df()
df.get_filtered_df()

,author(s),series,title,language,file,mirror1,mirror2,md5,topic,extension,size
0,"Hemingway, Ernest",,"Collected Works: A Farewell to Arms, The Sun A...",English,EPUB / 5.10 Mb,http://library.lol/fiction/774DCFA7F622848BBB2...,https://library.bz/fiction/edit/774DCFA7F62284...,774DCFA7F622848BBB2E642C8E51247D,fiction,epub,5.10 Mb
2,"Hemingway, Ernest",,The sun also rises,English,EPUB / 343 Kb,http://library.lol/fiction/2E71AD774DA0FD743AC...,https://library.bz/fiction/edit/2E71AD774DA0FD...,2E71AD774DA0FD743ACF5FFBEF5B26FE,fiction,epub,343 Kb
3,"Hemingway, Ernest",,The Sun Also Rises,English,EPUB / 167 Kb,http://library.lol/fiction/BEC6AF1BA4EF1FC35FF...,https://library.bz/fiction/edit/BEC6AF1BA4EF1F...,BEC6AF1BA4EF1FC35FF3A34B97D18A3B,fiction,epub,167 Kb
4,"Hemingway, Ernest",,The Sun Also Rises,English,EPUB / 372 Kb,http://library.lol/fiction/5624BF1DD3D8866AB12...,https://library.bz/fiction/edit/5624BF1DD3D886...,5624BF1DD3D8866AB129391E17360024,fiction,epub,372 Kb
5,"Hemingway, Ernest",,The sun also rises,English,EPUB / 166 Kb,http://library.lol/fiction/0FCDBD527D1F5EBA7A2...,https://library.bz/fiction/edit/0FCDBD527D1F5E...,0FCDBD527D1F5EBA7A292DA9404FB21F,fiction,epub,166 Kb
6,"Hemingway, Ernest",,The sun also rises,English,EPUB / 160 Kb,http://library.lol/fiction/7BABE541ED91A897CD1...,https://library.bz/fiction/edit/7BABE541ED91A8...,7BABE541ED91A897CD1526AA325A76FA,fiction,epub,160 Kb
7,"Hemingway, Ernest",,The Sun Also Rises,English,EPUB / 120 Kb,http://library.lol/fiction/B4DFAD3E301A0685884...,https://library.bz/fiction/edit/B4DFAD3E301A06...,B4DFAD3E301A0685884004B64681FB5B,fiction,epub,120 Kb
8,"Hemingway, Ernest",,The Sun Also Rises,English,EPUB / 119 Kb,http://library.lol/fiction/75AEF0305320F435AE2...,https://library.bz/fiction/edit/75AEF0305320F4...,75AEF0305320F435AE27B47FF8C01653,fiction,epub,119 Kb
9,"Hemingway, Ernest",,The sun also rises,English,EPUB / 160 Kb,http://library.lol/fiction/ECF02F0DCE5D96E9255...,https://library.bz/fiction/edit/ECF02F0DCE5D96...,ECF02F0DCE5D96E92558B5C3D19AFDA2,fiction,epub,160 Kb
10,"Hemingway, Ernest",,The Sun Also Rises,English,EPUB / 167 Kb,http://library.lol/fiction/AA9D5ACD5B4B19169C2...,https://library.bz/fiction/edit/AA9D5ACD5B4B19...,AA9D5ACD5B4B19169C277A62A07C49A1,fiction,epub,167 Kb


## How To: Use `EpubConvert` to convert Epub to PDF



In [ ]:
from autobook.epubconvert import EpubConvert

fileIn = "./books/Anthony Doerr - Cloud Cuckoo Land.epub"
fileOut = "./books/txt/cloud_cuckoo_land.txt"

convert = EpubConvert(fileIn).epub_to_txt(fileOut=fileOut)

## Version 1: Text Extraction

In [ ]:
import ebooklib
from autobook.ebook_wrapper import HTMLFilter
from ebooklib import epub

fileIn = "books/Anthony Doerr - Cloud Cuckoo Land.epub"
fileOut = "books/Anthony Doerr - Cloud Cuckoo Land.txt"

book = epub.read_epub(fileIn)
content = ""

for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        bodyContent = item.get_body_content().decode()
        f = HTMLFilter()
        f.feed(bodyContent)
        content += f.text

with open(fileOut, "w", encoding="utf-8") as fout:
    fout.write(content)

## Version 2: Text Extraction

In [ ]:
import ebooklib
from ebooklib import epub

file_name = "books/Anthony Doerr - Cloud Cuckoo Land.epub"

book = epub.read_epub(file_name)
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))
items_chapters = [item for item in items if "ch" in item.get_name()]

# for item in items:
#     if "ch" in item.get_name():
#         print(item.get_name())

In [ ]:
for item in items_chapters:
    print(item.get_name())

In [ ]:
from bs4 import BeautifulSoup


def chapter_to_str(chapter):
    soup = BeautifulSoup(chapter.get_body_content(), "html.parser")
    h1 = [header.get_text() for header in soup.find_all("h1")]
    h2 = [header.get_text() for header in soup.find_all("h2")]
    h3 = [header.get_text() for header in soup.find_all("h3")]
    if h1:  # only perform h1.append if h1 is not an empty list
        if not any("Chapter" in header for header in h1):
            h1 = ["Chapter " + header for header in h1]
        h1.insert(0, "      ")  # Adding spaces to the beginning of the list
        h1.append("     ")  # Adding spaces for a pause after `Chapter One`
    if h2:  # only perform h2.append if h2 is not an empty list
        h2.insert(0, "      ")  # Adding spaces to the beginning of the list
        h2.append(
            """


        """
        )  # Adding spaces for a pause after h2
    if h3:  # only perform h3.append if h3 is not an empty list
        h3.insert(0, "      ")  # Adding spaces to the beginning of the list
        h3.append(
            """


        """
        )  # Adding spaces for a pause after h3
    text = [para.get_text() for para in soup.find_all("p")]
    return "".join(h1 + h2 + h3 + text)


# chapter_to_str(items_chapters[16])


def chapters_to_txt(chapters: list, filename: str = None):
    texts = {}
    for chapter in chapters:
        texts[chapter.get_name()] = chapter_to_str(chapter)
    compiled_text = "\n\n\n".join(texts.values())
    with open(filename, "w", encoding="utf-8") as f:
        f.write(compiled_text)
    return texts


texts = chapters_to_txt(items_chapters)

In [ ]:
compiled_text = """


""".join(
    texts.values()
)

In [ ]:
with open("epubtxt.txt", "w", encoding="utf-8") as f:
    f.write(compiled_text)